In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_ori = pd.read_csv("watermelon_3.0.csv")

In [3]:
data_ori

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是
5,6,青绿,稍蜷,浊响,清晰,稍凹,软粘,0.403,0.237,是
6,7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,是
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否
9,10,青绿,硬挺,清脆,清晰,平坦,软粘,0.243,0.267,否


In [4]:
def data_prefix(data_ori):
    col_map = {}
    for col in data_ori.columns[:-1]:
        col_set = set(data_ori[col])
        if len(col_set) <= data_ori.shape[0] * 0.85:
            col_dict = {}
            i = 0
            for ele in col_set:
                col_dict[ele] = i
                i += 1
            col_map[col] = col_dict
    col_map[data_ori.columns[-1]] = {"是":1, "否":0}
    data_fix = data_ori[:]
    for col in data_ori.columns:
        if col in col_map:
            data_fix[col] = data_fix[col].map(col_map[col])
#     label = data_fix[data_fix.columns[-1]]
#     data = data_fix[data_fix.columns[:-1]]
    # return data, label, col_map
    return data_fix.drop(['编号'], axis=1), col_map

In [5]:
data, data_map = data_prefix(data_ori)

In [6]:
data_length = data.shape[0]
pos_prior = ((data['好瓜']==data_map['好瓜']['是']).sum()+1) / (data_length+len(data_map['好瓜']))
neg_prior = ((data['好瓜']==data_map['好瓜']['否']).sum()+1) / (data_length+len(data_map['好瓜']))
pos_num = (data['好瓜']==data_map['好瓜']['是']).sum()
neg_num = (data['好瓜']==data_map['好瓜']['否']).sum()

In [7]:
data_map

{'好瓜': {'否': 0, '是': 1},
 '敲声': {'沉闷': 0, '浊响': 1, '清脆': 2},
 '根蒂': {'硬挺': 0, '稍蜷': 1, '蜷缩': 2},
 '纹理': {'模糊': 2, '清晰': 1, '稍糊': 0},
 '脐部': {'凹陷': 0, '平坦': 2, '稍凹': 1},
 '色泽': {'乌黑': 2, '浅白': 1, '青绿': 0},
 '触感': {'硬滑': 0, '软粘': 1}}

In [8]:
pos_map = {}
neg_map = {}
for cols in data.columns:
    if cols=='好瓜':
        pass
    else:
        pos_map[cols] = {}
        neg_map[cols] = {}
        if cols in data_map:
            for feats in data_map[cols]:
                pos_f = ((data[data[cols]==data_map[cols][feats]]['好瓜']==
                          data_map['好瓜']['是']).sum()+1) / (pos_num + len(data_map[cols]))
                neg_f = ((data[data[cols]==data_map[cols][feats]]['好瓜']==
                          data_map['好瓜']['否']).sum()+1) / (neg_num + len(data_map[cols]))
                pos_map[cols][feats] = round(pos_f ,3)
                neg_map[cols][feats] = round(neg_f, 3)
        else:
            pos_map[cols]['mean'] = round(data[data['好瓜']==1][cols].mean(), 3)
            neg_map[cols]['mean'] = round(data[data['好瓜']==0][cols].mean(), 3)
            pos_map[cols]['std'] = round(data[data['好瓜']==1][cols].std(), 3)
            neg_map[cols]['std'] = round(data[data['好瓜']==0][cols].std(), 3)

In [9]:
pos_map

{'含糖率': {'mean': 0.279, 'std': 0.101},
 '密度': {'mean': 0.574, 'std': 0.129},
 '敲声': {'沉闷': 0.273, '浊响': 0.636, '清脆': 0.091},
 '根蒂': {'硬挺': 0.091, '稍蜷': 0.364, '蜷缩': 0.545},
 '纹理': {'模糊': 0.091, '清晰': 0.727, '稍糊': 0.182},
 '脐部': {'凹陷': 0.545, '平坦': 0.091, '稍凹': 0.364},
 '色泽': {'乌黑': 0.455, '浅白': 0.182, '青绿': 0.364},
 '触感': {'硬滑': 0.7, '软粘': 0.3}}

In [10]:
neg_map

{'含糖率': {'mean': 0.154, 'std': 0.108},
 '密度': {'mean': 0.496, 'std': 0.195},
 '敲声': {'沉闷': 0.333, '浊响': 0.417, '清脆': 0.25},
 '根蒂': {'硬挺': 0.25, '稍蜷': 0.417, '蜷缩': 0.333},
 '纹理': {'模糊': 0.333, '清晰': 0.25, '稍糊': 0.417},
 '脐部': {'凹陷': 0.25, '平坦': 0.417, '稍凹': 0.333},
 '色泽': {'乌黑': 0.25, '浅白': 0.417, '青绿': 0.333},
 '触感': {'硬滑': 0.636, '软粘': 0.364}}

In [11]:
a = pd.DataFrame(['青绿','蜷缩','浊响','清晰','凹陷','硬滑',0.697, 0.460]).T

In [12]:
a.columns = data.columns[:-1]

In [13]:
def NB(test):
    pos_prob = pos_prior
    neg_prob = neg_prior
    for cols in test:
        if 'mean' in pos_map[cols]: # 是连续值属性
            pos_feat_prob = 1 / ((np.sqrt(2*np.pi)) *  pos_map[cols]['std']) *  np.exp(-((test[cols][0] - pos_map[cols]['mean'])) ** 2 / (2*pos_map[cols]['std'] ** 2))
            pos_prob *= pos_feat_prob
            neg_feat_prob = 1 / ((np.sqrt(2*np.pi)) *  neg_map[cols]['std']) *  np.exp(-((test[cols][0] - neg_map[cols]['mean'])) ** 2 / (2*neg_map[cols]['std'] ** 2))
            neg_prob *= neg_feat_prob
        else:
            pos_prob *= pos_map[cols][test[cols][0]]
            neg_prob *= neg_map[cols][test[cols][0]]
    
    return '是' if pos_prob > neg_prob else '否'

In [14]:
NB(a)

'是'